<a href="https://colab.research.google.com/github/artms-18/tensorflow_fundamentals/blob/main/Intro_to_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to NLP Fundamentals in Tensorflow

NLP has the goal of deriving information out of natural language (could be sequences text or speech).

Another common term for NLP problems is sequence to sequence problems (seq2seq)

## Check for GPU

In [2]:
!nvidia-smi -L

GPU 0: Tesla K80 (UUID: GPU-4f7c324f-3787-426b-0297-872157d22345)


## Get helper functions

In [3]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

# Import a series of helper function for the notebook

from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

--2021-05-15 13:22:07--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2021-05-15 13:22:08 (52.3 MB/s) - ‘helper_functions.py’ saved [10246/10246]



## Get a text dataaset

The dataset we're going to be using is Kaggle's introduction to NLP dataset

In [4]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

unzip_data("nlp_getting_started.zip")

--2021-05-15 13:22:10--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.140.128, 108.177.15.128, 173.194.76.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.140.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.006s  

2021-05-15 13:22:11 (98.6 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



## Visualizing a text dataset

To visualie our text samples, we first have to read them in, one way to do so would be to use Python: 

Another way to do this is to use pandas.

(If text dataset is very large, cannot ise pandas (will tkaes of to much RAM).



In [5]:
import pandas as pd

train_df = pd.read_csv("train.csv")
test_df = pd.read_csv('test.csv')

train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [6]:
train_df['text'][0]

'Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all'

In [7]:
# Shuffle training dataframe

train_df_shuffled = train_df.sample(frac = 1, random_state = 42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [8]:
# What does the test dataframe look like?

test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [9]:
# How many examples of each class?
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [10]:
# How many total samples?
len(train_df), len(test_df)

(7613, 3263)

In [11]:
# Let's visualize some random training examples

import random

random_index = random.randint(0, len(train_df)-5) # create random indexes not higher than the total number of samples

for row in train_df_shuffled[['text', 'target']][random_index: random_index + 5].itertuples():
  _, text, target = row #gets rid of index
  print(f"Target: {target}", "(real disaster)" if target > 0 else "(not real disaster")
  print(f"Text:\n{text}\n")
  print("---\n")

Target: 1 (real disaster)
Text:
Fire crews evacuate passengers from a Gold Coast tram trapped when powerlines fell across a carriage. #TenNews 5pm http://t.co/hFyrloQY8q

---

Target: 1 (real disaster)
Text:
Debris found on Reunion Island comes from MH370: Malaysian PM http://t.co/f75qWyeeEC

---

Target: 0 (not real disaster
Text:
It's raining outside I'm burning my favorite candle and there's hot cocoa to sip. Nap time = awesomesauce.

---

Target: 0 (not real disaster
Text:
@58hif my trick is to think about nasty things

---

Target: 0 (not real disaster
Text:
Hunger hits me and I can't function probably wouldn't last long in famine zones***thank God***

---



### Split data into training and validation sets

In [12]:
from sklearn.model_selection import train_test_split

In [13]:
# Use train_test_split top split training data into training and validation sets

train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled['text'].to_numpy(), train_df_shuffled['target'].to_numpy(),
                                                                            test_size = 0.1, random_state = 42)

In [14]:
# Check the lengths

len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(6851, 6851, 762, 762)

In [15]:
# Check the first 10 samples

train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object), array([0, 

## Converting text into numbers

When dealing with a text problem, one of the first things you'll have to do before you can build a midek is to convert your text to numbers.

There are a few ways to do this, namely:
* Tokenization - direct mapping of  token( a toke could be a word or a character) to number
* embedding - create a matrix of feature vectors for each token the size of the feature vector can be defined and this embedding can be learned

### Text vectorization (tokenization)

In [16]:
train_sentences[:5]

array(['@mogacola @zamtriossu i screamed after hitting tweet',
       'Imagine getting flattened by Kurt Zouma',
       '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
       "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
       'Somehow find you and I collide http://t.co/Ee8RpOahPk'],
      dtype=object)

In [17]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

# Use the default TextVectorization parameters

text_vectorizer = TextVectorization(max_tokens = None, # how many words in the vocab (automatically add <OOV>)
                                    standardize = "lower_and_strip_punctuation",
                                    split = "whitespace", 
                                    ngrams = None,
                                    output_mode = "int", #how to map tokens to numbers
                                    output_sequence_length = None, #pads each sequence to the logest sequence
                                    pad_to_max_tokens = True)

In [18]:
# Find the average number of tokens (words) in the training tweets

round(sum(len(i.split()) for i in train_sentences)) / len(train_sentences)

14.901036345059115

In [19]:
# Setup text vectorization variables

max_vocab_length = 10000 #max num of words to have in our vocabulary
max_length = 15 # max length our seuqences will be (how many words in a tweet, amodel would see)

text_vectorizer = TextVectorization(max_tokens = max_vocab_length, output_mode= 'int', output_sequence_length = max_length)

In [20]:
#fit the text vectorier to the training text

text_vectorizer.adapt(train_sentences)

In [21]:
# Create a sample sentence and tokenize it

sample_sentence = "Theres a flood in my street!"

text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [22]:
# Choose a random sentence from the training dataset and tokenize iot

random_sentence = random.choice(train_sentences)
print(f"Original text:\n {random_sentence}\n\nVectorized version:")
text_vectorizer([random_sentence])

Original text:
 Three Israeli soldiers wounded in West Bank terrorist attack via /r/worldnews http://t.co/9TyucdWh3g

Vectorized version:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[ 564,  602, 1362,  269,    4,  695, 1896,  309,  113,   49, 3452,
           1,    0,    0,    0]])>

In [23]:
# Get the unique words in the vocabulary

words_in_vocab = text_vectorizer.get_vocabulary() #get all of the unique words in the training data
top_5_words = words_in_vocab[:5] # get the most common words
bottom_5_words = words_in_vocab[-5:] #get the least common words
print(f"Number of words in vocab: {len(words_in_vocab)}")
print(f"5 most comon words: {top_5_words}")

Number of words in vocab: 10000
5 most comon words: ['', '[UNK]', 'the', 'a', 'in']


### Creating an Embedding using an Embedding Layer

To make our embedding, we're going to use tensoflows embedding layer

The parameters we actually care about:
* `input_dim` = the size of our vocabulary
* `output_dim` = the size of the output embedding vector, for example, a value of 100 would mean each token gets represented by a vecotr 100 long
* `input_length` = length of the sequences being passed to the embedding layer

In [24]:
from tensorflow.keras import layers

# in keras, wordks best when numbers are divisible by eight

embedding = layers.Embedding(input_dim = max_vocab_length,
                             output_dim = 128,
                             input_length = max_length
                             )
embedding

In [25]:
# Get a random sentence from the training set

random_sentence = random.choice(train_sentences)
print(f"Original text:\n {random_sentence}\n\nEmbedded version:")

# Embed the random sentence (turn into dense vecotrs of fixed size)
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text:
 @CarsonRex @SpaceAngelSeven Check out this #rockin preview of @ClaytonBryant Danger Zone Coming soon! https://t.co/P0fiZxmN5r #ArtistsUnited

Embedded version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.02523334,  0.03493084, -0.04796112, ...,  0.00853187,
         -0.03687423, -0.01564975],
        [ 0.02128121,  0.01815982, -0.01483806, ...,  0.01610202,
         -0.00082193,  0.04479123],
        [-0.03298303,  0.0289293 ,  0.04010714, ...,  0.00959096,
         -0.04420332, -0.04931916],
        ...,
        [-0.04974687, -0.03320911,  0.02238791, ..., -0.02066599,
         -0.02062473,  0.02362857],
        [ 0.02523334,  0.03493084, -0.04796112, ...,  0.00853187,
         -0.03687423, -0.01564975],
        [-0.02592582, -0.0493325 ,  0.04398059, ..., -0.04771078,
          0.03944752, -0.03976402]]], dtype=float32)>

In [26]:
# Chekc out a single token's embedding

sample_embed[0][0], sample_embed[0][0].shape, random_sentence[0]

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([ 0.02523334,  0.03493084, -0.04796112,  0.01927746,  0.00765488,
         0.04679333, -0.03514147, -0.02424946,  0.01218501,  0.01936242,
        -0.0049302 , -0.01804946, -0.02241603, -0.04510611, -0.03608831,
         0.02249565,  0.00688655, -0.01790222,  0.02551429,  0.03070218,
         0.02970531, -0.01329334,  0.01394827,  0.02314376, -0.0461725 ,
        -0.04819785,  0.01322627,  0.00382049, -0.01006036,  0.00255274,
         0.00314039,  0.02688238,  0.03423582,  0.03898888,  0.03463617,
         0.00329806, -0.00075484, -0.03316444, -0.01125193,  0.0258862 ,
        -0.02649372,  0.0486881 ,  0.04760689, -0.03956348, -0.04299253,
        -0.00200772,  0.03094386, -0.00972052, -0.02930557,  0.04761081,
         0.01266747,  0.0371767 , -0.00709808,  0.0173412 ,  0.00800402,
         0.01247799, -0.04360623,  0.04878442, -0.03811219, -0.01582158,
        -0.03138728, -0.03096091,  0.04720249,  0.03125962, -0.04959358,
  

## Modelling a text dataset (running a series of experiments)

Now we're got a wat to turn our text srequences into numbers, it's time to start building a series of modelling experiments, start from a baseline and move on from there 

* Model 0: Naive Bayes (baseline)
* Model 1: Feed-forward neual network (dense model)
* Model 2: GRU model (RNN)
* Model 4: Bidirectional-LSTM
* Model 5: 1d Convolutional Neural Network (CNN)
* Model 6: Tensorflow Hub Pretrained Geature Extractor (using transfer learning for NLP)
* Model 7: Same as model 6 with 10% of training data

How are we going to approach all of these?

Use the standard steps in modelling with tensorflow:

* Create a model
* Build a model
* Compile a model
* Fit the model

### Model 0: Getting a baseline

As with all machine learning modelling experiments it's important to create a baseline model so u got a benchmark for future experiments to build upon

To create our baseline, we'll use SKlearns Multinomial Naive Bayes using the TF-IDF formula to convert our words to numbers

> **Note:** It's common practive to use non-DL algorithms as a baseline because of their speed, then later using DL to improve

In [27]:

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# Create tokenization and modelling pipeline

model_0 = Pipeline([
                    ("tfidf", TfidfVectorizer()),
                    ("clf", MultinomialNB())
])

# Fit the pipeline to the training data

model_0.fit(train_sentences, train_labels)

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('clf',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [28]:
# Evaluate our baseline model 

baseline_score = model_0.score(val_sentences, val_labels)
print(f"Our baseline model achieves an accuracy of: {baseline_score*100:.2f}%")

Our baseline model achieves an accuracy of: 79.27%


In [32]:
# Make predictions

baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

### Creating an evaluation function for our model experiments

We could evaluate model's predictin with different methods every time, but this could be cumbersome and could easily be fixed with a function

Let's create one to compare our model's predictions with the truth labels using the following metrics:

* Accuracy
* Precision
* Recall
* F1-score

In [33]:
# Function to evaluate: accuracty, precision, recall, f1-score

from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
  """
  Calulates model accuracy, precision, recall and f1 score of a binary classification model.
  """
  #Calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  #Calculate model precision, recall ,and f01 score using "weighted" average
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average = 'weighted')
  model_results = {"accuracy": model_accuracy,
                   "precision": model_precision,
                   "recall": model_recall,
                   "f1": model_f1}
  return model_results

In [37]:
baseline_results = calculate_results(y_true = val_labels, y_pred = baseline_preds)

baseline_results

{'accuracy': 79.26509186351706,
 'f1': 0.7862189758049549,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706}

### Model 1: A simple dense model

In [38]:
# Create a tensorboard callback (need to create a new one for each model)

from helper_functions import create_tensorboard_callback

# Create a directory to save TensorBoard logs

SAVE_DIR = "model_logs"

In [65]:
from tensorflow.keras import layers

inputs = layers.Input(shape = (1,), dtype = tf.string) # inputs are 1-dimensional
x = text_vectorizer(inputs) # turn the input text into numbers
x = embedding(x) # create an embedding of the numberized imputs
x = layers.GlobalAveragePooling1D()(x) # condense the feature vector for each token to one vector
#x = layers.GlobalMaxPool1D()(x)
outputs = layers.Dense(1, activation = "sigmoid")(x) # Create the output layer, want binary output so use sigmoid

model_1 = tf.keras.Model(inputs, outputs, name = "model_1_dense")

In [59]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
text_vectorization_1 (TextVe (None, 15)                0         
_________________________________________________________________
embedding (Embedding)        (None, 15, 128)           1280000   
_________________________________________________________________
global_average_pooling1d_1 ( (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 129       
Total params: 1,280,129
Trainable params: 1,280,129
Non-trainable params: 0
_________________________________________________________________


In [60]:
# Compile mode

model_1.compile(loss = "binary_crossentropy", optimizer = tf.keras.optimizers.Adam(), metrics = ["accuracy"])

In [61]:
# Fit the mode
model_1_history = model_1.fit(x = train_sentences, y = train_labels, epochs = 15, validation_data = (val_sentences, val_labels),
                              callbacks = [create_tensorboard_callback(dir_name = SAVE_DIR, experiment_name = "model_1_dense")])

Saving TensorBoard log files to: model_logs/model_1_dense/20210515-140739
Epoch 1/15
215/215 [==============================] - 5s 19ms/step - loss: 0.5663 - accuracy: 0.8197 - val_loss: 0.4910 - val_accuracy: 0.7769
Epoch 2/15
215/215 [==============================] - 3s 16ms/step - loss: 0.3487 - accuracy: 0.8793 - val_loss: 0.4522 - val_accuracy: 0.7848
Epoch 3/15
215/215 [==============================] - 3s 16ms/step - loss: 0.2610 - accuracy: 0.9091 - val_loss: 0.4515 - val_accuracy: 0.7900
Epoch 4/15
215/215 [==============================] - 3s 16ms/step - loss: 0.2256 - accuracy: 0.9214 - val_loss: 0.4639 - val_accuracy: 0.7887
Epoch 5/15
215/215 [==============================] - 3s 16ms/step - loss: 0.1880 - accuracy: 0.9367 - val_loss: 0.4808 - val_accuracy: 0.7795
Epoch 6/15
215/215 [==============================] - 3s 16ms/step - loss: 0.1706 - accuracy: 0.9442 - val_loss: 0.5034 - val_accuracy: 0.7874
Epoch 7/15
215/215 [==============================] - 3s 16ms/step -

In [62]:
# Check the results

model_1.evaluate(val_sentences, val_labels)

24/24 [==============================] - 0s 4ms/step - loss: 0.7696 - accuracy: 0.7730


[0.7695834636688232, 0.7729659080505371]

In [63]:
model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs.shape

(762, 1)

In [64]:
# look at a single prediction

model_1_pred_probs[0] # anything below 0.5 = disaster, abouve = non-=disaster

array([0.10121736], dtype=float32)

In [66]:
# look at the first 10 predictions

model_1_pred_probs[:10]

array([[1.0121736e-01],
       [7.8011036e-01],
       [9.9973923e-01],
       [6.2192809e-02],
       [9.9454750e-04],
       [9.9302447e-01],
       [7.4617761e-01],
       [9.9999940e-01],
       [9.9996567e-01],
       [5.4841375e-01]], dtype=float32)

In [69]:
# Convert model prediction probablilities to label format

model_1_preds = tf.squeeze(tf.round(model_1_pred_probs))

model_1_preds[:20]

<tf.Tensor: shape=(20,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1.], dtype=float32)>

In [71]:
# calculate our model_1 results

model_1_results = calculate_results(y_true = val_labels, y_pred = model_1_preds)

model_1_results

{'accuracy': 77.29658792650919,
 'f1': 0.7709704727215051,
 'precision': 0.7745161732477092,
 'recall': 0.7729658792650919}

In [72]:
baseline_results

{'accuracy': 79.26509186351706,
 'f1': 0.7862189758049549,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706}

In [73]:
import numpy as np

np.array(list(model_1_results.values())) > np.array(list(baseline_results.values()))

array([False, False, False, False])